<a href="https://colab.research.google.com/github/450586509/reinforcement-learning-practice/blob/master/05_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this if in COLAB
!pip install --upgrade https://github.com/Theano/Theano/archive/master.zip
!pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip
    
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week5_explore/bayes.py
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week5_explore/action_rewards.npy
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week5_explore/all_states.npy

     - 29.9MB 5.7MB/s
  Created wheel for Theano: filename=Theano-1.0.4+unknown-cp36-none-any.whl size=2666479 sha256=c995836832df81f1637b03775d1ccd79e2354235eacd74464026c2bade92ef2d
  Stored in directory: /tmp/pip-ephem-wheel-cache-exs9j4q3/wheels/33/73/96/0ed263c62a86e2485ea634e0d3ae8169d50fd66e3b252541db
Successfully built Theano
  Found existing installation: Theano 1.0.4
    Uninstalling Theano-1.0.4:
      Successfully uninstalled Theano-1.0.4
     - 2.1MB 27.4MB/s
  Created wheel for Lasagne: filename=Lasagne-0.2.dev1-cp36-none-any.whl size=122750 sha256=00aa840d838b0656f976ca992d57d91808ce332d514ee919a27dc10dc8f9e98f
  Stored in directory: /tmp/pip-ephem-wheel-cache-jutld35p/wheels/ca/4a/00/87f1777b229481fe76562df7c0cfb993bc88ed0cc37e3f0ed4
Successfully built Lasagne


In [0]:
from abc import ABCMeta, abstractmethod, abstractproperty
import enum

import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

import pandas

from matplotlib import pyplot as plt
%matplotlib inline

## Bernouli Bandit问题
* 实现几个简单的探索策略，解决这个问题。
* bandit有K个动作，每一个动作有 $0 \le \theta_k \le 1$的概率获得reward=1的奖励，这个概率分布对于agent是未知的，但是固定不变的。agent的优化目标最小化下面的等式：$$\rho = T\theta^* - \sum_{t=1}^T r_t$$T是选择次数,$\theta^* = \max_k\{\theta_k\}$
* 


In [0]:
class BernoulliBandit:
    def __init__(self, n_actions=5):
        self._probs = np.random.random(n_actions)

    @property
    def action_count(self):
        return len(self._probs)

    def pull(self, action):
        if np.any(np.random.random() > self._probs[action]):
            return 0.0
        return 1.0

    def optimal_reward(self):
        """ Used for regret calculation
        """
        return np.max(self._probs)

    def step(self):
        """ Used in nonstationary version
        """
        pass

    def reset(self):
        """ Used in nonstationary version
        """
class AbstractAgent(metaclass=ABCMeta):
    def init_actions(self, n_actions):
        self._successes = np.zeros(n_actions)
        self._failures = np.zeros(n_actions)
        self._total_pulls = 0

    @abstractmethod
    def get_action(self):
        """
        Get current best action
        :rtype: int
        """
        pass

    def update(self, action, reward):
        """
        Observe reward from action and update agent's internal parameters
        :type action: int
        :type reward: int
        """
        self._total_pulls += 1
        if reward == 1:
            self._successes[action] += 1
        else:
            self._failures[action] += 1

    @property
    def name(self):
        return self.__class__.__name__


class RandomAgent(AbstractAgent):
    def get_action(self):
        return np.random.randint(0, len(self._successes))

In [0]:
###

#### epsilon-greedy agent

**for** $t = 1,2,...$ **do**

&nbsp;&nbsp; **for** $k = 1,...,K$ **do**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\hat\theta_k \leftarrow \alpha_k / (\alpha_k + \beta_k)$

&nbsp;&nbsp; **end for** 

&nbsp;&nbsp; $x_t \leftarrow argmax_{k}\hat\theta$ with probability $1 - \epsilon$ or random action with probability $\epsilon$

&nbsp;&nbsp; Apply $x_t$ and observe $r_t$

&nbsp;&nbsp; $(\alpha_{x_t}, \beta_{x_t}) \leftarrow (\alpha_{x_t}, \beta_{x_t}) + (r_t, 1-r_t)$

**end for**

Implement the algorithm above in the cell below:


In [1]:
class EpsilonGreedyAgent(AbstractAgent):
    def __init__(self, epsilon=0.01):
        self._epsilon = epsilon
    def best_action(self):
      max_p = -1
      best_action = -1
      for action, (a, b) in enumerate(zip(self._successes, self._failures)):
        if a + b == 0:
          p = 0
        else:
          p = (1.0 * a) / (a + b)
        if max_p < p :
          max_p = p
          best_action = action
      return best_action
          

    def get_action(self):
        # YOUR CODE HERE
        # best_action
        best_action = self.best_action()
        is_choice_random_action = np.random.choice([True, False], p=[self._epsilon, 1-self._epsilon])
        random_action = np.random.choice(range(len(self._successes)))
        if is_choice_random_action:
          return random_action
        return best_action

    @property
    def name(self):
        return self.__class__.__name__ + "(epsilon={})".format(self._epsilon)

NameError: ignored

In [0]:
]